In [ ]:
! pip install cobra

In [ ]:
!pip install reframed

In [ ]:
import cobra
import reframed
from google.colab import files
import numpy as np
import pandas as pd
import sympy
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import scipy
import os

In [ ]:
cobra.Configuration().solver = "glpk_exact"

In [ ]:
files.upload()

Saving data sheet 1.xlsx to data sheet 1.xlsx


{'data sheet 1.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00\x14\xd2\x12M\x17\x02\x00\x00\\\r\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [ ]:
df_gpr=pd.read_excel('/content/data sheet 1.xlsx',sheet_name='grRule')
df_gpr2=df_gpr[['Reaction Name','GPR rules']].dropna()
df_gprarray=np.array(df_gpr2)
print(df_gpr.shape,df_gprarray.shape)

(671, 7) (461, 2)


In [ ]:
cb_modelglio=cobra.io.load_matlab_model('/content/tissue.mat')
cb_modelglio

Name,tissueModel
Memory address,0x07f463c8ca240
Number of metabolites,548
Number of reactions,625
Number of groups,0
Objective expression,1.0*n_E24 - 1.0*n_E24_reverse_e44f2
Compartments,cytosol


In [ ]:
matrix=cobra.util.create_stoichiometric_matrix(cb_modelglio,array_type='dense',dtype=np.float64)
print(matrix,matrix.shape)

[[ -1.   0.   0. ...   0.   0.   0.]
 [ -1.   0.  -1. ...   0.   0.   0.]
 [  1.  -1.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0. -24.   0.]
 [  0.   0.   0. ...   1.  24.   0.]
 [  0.   0.   0. ...   0.   1.  -1.]] (548, 625)


In [ ]:
x=cb_modelglio.objective.expression
y=cobra.util.solver.linear_reaction_coefficients(cb_modelglio)
print(x,'\n',y)

1.0*n_E24 - 1.0*n_E24_reverse_e44f2 
 {<Reaction n_E24 at 0x7f463a3bb940>: 1.0}


In [ ]:
#df_flux=pd.read_excel('/content/data sheet 1.xlsx',sheet_name='Model constraints')

#df_fluxarr=np.array(df_flux)

#for i in cb_modelglio.reactions:
  #for j in range(df_fluxarr.shape[0]):
    #if i.name==df_fluxarr[j,0]:
      #i.upper_bound=df_fluxarr[j,2]
      #i.lower_bound=df_fluxarr[j,3]

#cb_modelglio.reactions[97]

Reaction identifier,n_R98
Name,R98
Memory address,0x07f463ad84390
Stoichiometry,m_Glutamine[cytosol] --> m_Glutamine_N[cytosol] Glutamine --> Glutamine_N
GPR,
Lower bound,0.0048
Upper bound,0.0048


In [ ]:
fba=cb_modelglio.optimize()
fba

,fluxes,reduced_costs
n_R1,0.000000,0.000000
n_R2,-136.059095,0.000000
n_R3,143.731408,0.000000
n_R4,143.731408,0.000000
n_R5,124.397738,0.000000
...,...,...
n_E21,37.808333,0.000000
n_E22,1000.000000,0.083333
n_E23,-1000.000000,0.000000
n_E24,41.666667,0.000000


In [ ]:
for i in cb_modelglio.reactions:
  for j in range(df_gprarray.shape[0]):
    if df_gprarray[j,0]==i.name:
      i.gene_reaction_rule=df_gprarray[j,1]

      
cb_modelglio.repair()


#cb_modelglio.genes=cb_modelglio.genes[569:]
#cb_modelglio.repair()


In [ ]:
for i in cb_modelglio.genes:
  i.name=i.id
cb_modelglio.repair()
len(cb_modelglio.genes)

570

In [ ]:
#df=pd.read_excel('/content/avg.xlsx',sheet_name='Sheet1')
#df=df.drop(df.columns[0],axis=1)
#df_arr=np.array(df)
#print(df_arr)

[['COL3A1' 21.91]
 ['TNC' 11.73]
 ['COL4A1' 10.705]
 ...
 ['GABRA5' -21.25]
 ['SST' -21.55]
 ['SYT1' -30.725]]


In [ ]:
#my_dict={}

#a=[j.name for j in cb_modelglio.genes]

#for i in range(df_arr.shape[0]):
  #if df_arr[i,0] in a:
    #my_dict[df_arr[i,0]]=df_arr[i,1]

#for s in a:
  #if s not in my_dict.keys():
   # my_dict[s]=-1


#print(my_dict)

#out=pd.DataFrame({'gene':my_dict.keys(),'fold_change':my_dict.values()})
#out.to_excel('expression.xlsx')

{'HMOX1': 5.86, 'VCAN': 4.51, 'HK2': 3.49, 'IDH1': 2.68, 'INPPL1': 2.58, 'GPX1': 2.3, 'UPP1': 2.15, 'PGM2': 2.135, 'ATP6V0E1': 1.993333333333333, 'IMPAD1': 1.99, 'LSG1': 1.775, 'NME4': 1.77, 'CANT1': 1.736666666666667, 'NME1': 1.71, 'UCKL1': 1.695, 'ATP1B3': 1.64, 'ENTPD1': 1.603333333333333, 'PNP': 1.6, 'ENO1': 1.56, 'NME6': 1.56, 'PGM1': 1.54, 'PLCG1': 1.53, 'PI4K2B': 1.53, 'ACAA1': 1.51, 'PFKL': 1.48, 'SMPD1': 1.47, 'NME3': 1.46, 'NDUFA9': 1.45, 'NDUFA11': 1.42, 'IDH2': 1.42, 'SLC27A2': -1.415, 'HK1': -1.43, 'PIP4K2B': -1.446666666666667, 'DGKA': -1.45, 'PFKP': -1.48, 'UPP2': -1.5, 'NDUFA10': -1.52, 'ATP6V1E1': -1.52, 'ATP6V1H': -1.56, 'NOS1': -1.586, 'PLCD4': -1.59, 'PLCH2': -1.61, 'PLCB4': -1.63, 'ACSL4': -1.65, 'PLCH1': -1.66, 'NDUFB8': -1.665, 'HMGCR': -1.675, 'ATP6V1B2': -1.7, 'INPP4A': -1.796, 'ATP6V1C1': -1.8525, 'GLS': -1.884285714285714, 'ENTPD3': -1.91, 'GLS2': -1.96, 'INPP5A': -2.085, 'UQCRB': -2.09, 'SYNJ2': -2.115, 'DGKE': -2.17, 'DGKZ': -2.3, 'TYRP1': -2.3, 'GAD2': -2.

In [ ]:
def adjancey_matrix(H,D_v,W):
  W=np.diag(W)
  A = np.subtract(H @ W @ H.T ,D_v)
  return A

def normalized_laplacian(D_v,D_e,H,W):
  A=adjancey_matrix(H,D_v,W)
  #phi = np.sqrt(np.linalg.inv(D_v)) @ H @ W @ np.linalg.inv(D_e) @ H.T @ np.sqrt(np.linalg.inv(D_v))
  #l=np.eye(phi.shape[0]) - phi
  phi= np.sqrt(np.linalg.inv(D_v)) @ A @ np.sqrt(np.linalg.inv(D_v))
  l=1/2*(np.eye(phi.shape[0]) - phi)
  return l

def spectra_de(l,k):
  eigen_val,eigen_vec=np.linalg.eig(l)

  eig_vec_sorted=eigen_vec[:,eigen_val.argsort()]

  eig_vec_desired=eig_vec_sorted[:,:k]

  return eig_vec_desired


def clustering(l,k):
  x=spectra_de(l,k)
  m=KMeans(n_clusters=k).fit(x)
  ans=m.labels_
  return ans

def get_incidencematrix(stiochiometry_matrix):

  a=stiochiometry_matrix

  for i in range(a.shape[0]):
    for j in range(a.shape[1]):

      if a[i,j] != 0:
        a[i,j]=1

      else:
        a[i,j]=0
  return a

def Deg_v(stiochiometry_matrix,W):
  h=get_incidencematrix(stiochiometry_matrix)
  ans= h @ W.T
  ans_diag=np.diag(ans.T)
  return ans_diag

def Deg_e(stiochiometry_matrix):
  h=get_incidencematrix(stiochiometry_matrix)
  ans=np.sum(h,axis=0)
  ans_diag=np.diag(ans)
  return ans_diag

In [ ]:
a=fba.fluxes.values
a=np.abs(a)
for i in range(a.shape[0]):
  if a[i]==0:
    a[i]=0.000001

print(a)

[1.00000000e-06 1.36059095e+02 1.43731408e+02 1.43731408e+02
 1.24397738e+02 4.08024397e+02 4.08024397e+02 4.08024397e+02
 4.08024397e+02 4.08024397e+02 1.93614806e+02 1.00626416e+02
 1.34101019e+02 1.00000000e-06 1.00000000e-06 1.00000000e-06
 4.19685754e+02 4.19685754e+02 4.19685754e+02 2.79790503e+02
 1.39895251e+02 1.39895251e+02 1.39895251e+02 1.39895251e+02
 6.84912273e+01 6.84912273e+01 6.84912273e+01 1.00000000e-06
 1.00000000e-06 1.00000000e-06 6.84912273e+01 1.00000000e-06
 1.00000000e-06 1.94164090e+02 1.00000000e-06 1.00000000e-06
 1.00000000e-06 1.00000000e-06 1.00000000e-06 1.00000000e-06
 2.27078920e+02 5.15280720e+02 5.15280720e+02 2.88201801e+02
 1.00000000e+03 7.16832708e+00 1.06344622e+01 1.23406400e+00
 1.23406400e+00 1.00000000e-06 7.16832708e+00 7.16832708e+00
 7.16832708e+00 7.16832708e+00 7.16832708e+00 1.00000000e-06
 1.00000000e-06 1.00000000e-06 1.00000000e-06 1.00000000e-06
 1.78027892e+01 1.78027892e+01 1.78027892e+01 1.18685262e+01
 5.93426308e+00 5.934263

In [ ]:
h=get_incidencematrix(matrix)
D_v=Deg_v(matrix,a)
D_e=Deg_e(matrix)
normalized=normalized_laplacian(D_v,D_e,h,a)
ans=clustering(normalized,5)
 
print(ans.shape)

(548,)


In [ ]:
len(cb_modelglio.metabolites)

548

In [ ]:
a=[i.name for i in cb_modelglio.metabolites]

b=list(ans)

s=dict(zip(a,b))

df_re=pd.DataFrame({'metabolites':s.keys(),'clusters':s.values()})

df_re.to_excel('output2.xlsx')

df_re

,metabolites,clusters
0,Glucose_A,0
1,ATP_A,0
2,Glucose-6-phosphate_A,0
3,ADP_A,0
4,Fructose-6-phosphate_A,0
...,...,...
543,RNA,0
544,lipid_WM,0
545,ATP,0
546,ADP,0


In [ ]:
cobra.io.save_matlab_model(cb_modelglio,'iMS570g3.mat')